# Welcome to Automatic Thai Question Generation from Context!
This is notebook detailing how to finetune **mT5 for question-generation in the Thai language!** Specifically this notebook will train the **mT5-small-thai-e2e-qg-aug-numsep.**


First, we will mount our google drive so our models don't get deleted (╯°□°)╯︵ ┻━┻ 


In [1]:
from google.colab import drive

drive.mount("/content/drive")


Mounted at /content/drive


# Setup

Install dependencies that we need in order to run/train our model

## Install dependencies

In [1]:
# Remove "sample_data" in colab
!rm -rf sample_data
# Solve some protobuf problems
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION='python'

## Install dependencies
%pip install torch ijson pandas torchmetrics lightning-bolts transformers sentencepiece protobuf beautifulsoup4 pytorch-lightning 
%pip install pythainlp epitran
%pip install -U nltk
%pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 33.9 MB/s 
     |████████████████████████████████| 419 kB 63.9 MB/s 
     |████████████████████████████████| 316 kB 51.3 MB/s 
     |████████████████████████████████| 4.2 MB 56.1 MB/s 
     |████████████████████████████████| 1.2 MB 57.4 MB/s 
     |████████████████████████████████| 585 kB 68.2 MB/s 
     |████████████████████████████████| 596 kB 69.2 MB/s 
     |████████████████████████████████| 140 kB 77.7 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 61.2 MB/s 
     |████████████████████████████████| 144 kB 66.6 MB/s 
     |████████████████████████████████| 271 kB 70.3 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling 

And then login to wandb to keep track of our training

In [ ]:
!wandb login

Then we will import all the things we need

In [2]:
import math
import urllib.request
import os
import ijson
import json
import re
from typing import Optional

import torch

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger


from zipfile import ZipFile
from bs4 import BeautifulSoup
from transformers import (
    MT5ForConditionalGeneration,
    MT5TokenizerFast,
)


# Gather & Process datasets
- xquad-thai
- iapp-wiki-qa-dataset
- thaiqa

In [3]:
def download_dataset(url, file_name):
    urllib.request.urlretrieve(
        url,
        os.path.join("dataset/", file_name),
        reporthook=(
            lambda count, block, total: print(
                f"Downloading {file_name}: {math.floor((count * block) / total * 100)}%",
                end="\r",
            )
        ),
    )
    print(f"Downloaded {file_name} from {url}")


# Check if the dataset already exists
if not (
    os.path.exists("dataset/xquad.json")
    and os.path.exists("dataset/iapp-thai-wikipedia-qa.json")
):
    os.mkdir("dataset")
    # Download all datasets
    download_dataset(
        "https://github.com/deepmind/xquad/raw/master/xquad.th.json", "xquad.json"
    )
    download_dataset(
        "https://raw.githubusercontent.com/iapp-technology/iapp-wiki-qa-dataset/main/iapp-thai-wikipedia-qa-1961-docs-9170-questions.json",
        "iapp-thai-wikipedia-qa.json",
    )
    download_dataset(
        "https://github.com/PyThaiNLP/thaiqa_squad/raw/main/data.zip", "thaiqa.zip"
    )
    with ZipFile("dataset/thaiqa.zip") as zipfile:
        os.mkdir("dataset/thaiqa")
        zipfile.extractall("dataset/thaiqa/")


Downloaded xquad.json from https://github.com/deepmind/xquad/raw/master/xquad.th.json
Downloaded iapp-thai-wikipedia-qa.json from https://raw.githubusercontent.com/iapp-technology/iapp-wiki-qa-dataset/main/iapp-thai-wikipedia-qa-1961-docs-9170-questions.json
Downloaded thaiqa.zip from https://github.com/PyThaiNLP/thaiqa_squad/raw/main/data.zip


Before we load our data into memory, we need to augment data since our dataset is very small.

In [33]:
def augment_data_xquad(source_list, target_list, context, qas):
    for amt_questions in range(1, len(qas) + 1):
        source_text = f"สร้าง {amt_questions} คำถาม: {context}"
        target_text = ""

        for index, qa in enumerate(qas[:amt_questions]):
            target_text += (
                f" <{index + 1}> {qa['question'].strip()} A: {qa['answers'][0]['text'].strip()}"
            )

        source_list.append(source_text.strip())
        target_list.append(target_text.strip())


def augment_data_iapp(source_list, target_list, context, qas):
    filtered_qas = list(filter(lambda x: len(x["a"]) != 0 and len(x["q"]) != 0, qas))
    for amt_questions in range(1, len(filtered_qas) + 1):
        source_text = f"สร้าง {amt_questions} คำถาม: {context}"
        target_text = ""

        for index, qa in enumerate(filtered_qas[:amt_questions]):
            target_text += f" <{index + 1}> {qa['q'].strip()} A: {qa['a'][0].strip()}"

        source_list.append(source_text.strip())
        target_list.append(target_text.strip())


def augment_data_thaiqa(source_list, target_list, context, qas):
    for amt_questions in range(1, len(qas) + 1):
        source_text = f"สร้าง {amt_questions} คำถาม: {context}"
        target_text = ""

        for index, qa in enumerate(list(qas.iterrows())[:amt_questions]):
            target_text += f" <{index + 1}> {qa[1]['question'].strip()} A: {qa[1]['answer'].strip()}"

        source_list.append(source_text.strip())
        target_list.append(target_text.strip())


In [34]:
# This list will store all the Q&A
source_list = []
target_list = []

# Start cleaning data
squad = open(os.path.join("dataset/", "xquad.json"))
iapp = open(os.path.join("dataset/", "iapp-thai-wikipedia-qa.json"))
iapp_keys = open(os.path.join("dataset/", "iapp-thai-wikipedia-qa.json"))
thaiqa = open(os.path.join("dataset/thaiqa/data/train.jsonl"))

squad_json = ijson.items(squad, "data.item")
iapp_json = json.load(iapp)
iapp_keys = ijson.kvitems(iapp_keys, "db")
thaiqa_df = pd.read_json(thaiqa, lines=True)

# Get data from xquad
for obj in squad_json:
    paragraphs = obj["paragraphs"]
    for p in paragraphs:
        context = p["context"]
        qas = [p for p in p["qas"] if len(p) > 0]

        augment_data_xquad(source_list, target_list, context, qas)

# Get dataset from iapp
for key in iapp_keys:
    try:
        obj = iapp_json["db"][key[0]]
        context = obj["detail"]
        qas = obj["QA"]
        target_text = ""

        augment_data_iapp(source_list, target_list, context, qas)

    except KeyError as e:
        # Due to the dataset, there will always be a keyerror on "detail" which is the dataset's metadata
        if str(e) != "'detail'":
            print(f"KeyError: {e}")

# Get data from thaiqa
article_ids = set(thaiqa_df["article_id"])
for id in article_ids:
    questions = thaiqa_df[thaiqa_df["article_id"] == id]

    # Remove html markup
    soup = BeautifulSoup(questions["context"].iloc[0])

    # Remove parenthesis because some are empty
    context = re.sub(r"\(\)", "", soup.text)

    # Remove double spaces resulting from removing parenthesis
    context = re.sub(r"\s\s+", " ", context)

    augment_data_thaiqa(source_list, target_list, context, questions)

dataframe = pd.DataFrame({"source_text": source_list, "target_text": target_list})


And then split our dataframe into train, valid, and test sets.

In [5]:
index_train_split = math.floor(dataframe.shape[0] * 0.8)
train_df, valid_test = (
    dataframe.iloc[
        :index_train_split,
    ],
    dataframe.iloc[
        index_train_split:,
    ],
)

index_test_split = math.floor(valid_test.shape[0] * 0.5)
valid_df, test_df = (
    valid_test.iloc[
        :index_test_split,
    ],
    valid_test.iloc[
        index_test_split:,
    ],
)


# Training
Now we can finally get to the nitty-gritty and start defining our training loops

In [6]:
pl.seed_everything(16)
torch.cuda.empty_cache()


class MT5Dataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.data = df.reset_index()
        self.tokenizer = tokenizer
        self.source_max_len = 1024
        self.target_max_len = 1024

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_row = self.data.iloc[idx]
        source, target = data_row["source_text"], data_row["target_text"]

        source_encoding = self.tokenizer(
            source,
            padding="max_length",
            max_length=self.source_max_len,
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        target_encoding = self.tokenizer(
            target,
            padding="max_length",
            max_length=self.target_max_len,
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        # Ensure labels are correct (see huggingface T5 training documentation)
        labels = target_encoding.input_ids
        labels[labels == self.tokenizer.pad_token_id] = -100

        return dict(
            input_ids=source_encoding.input_ids.flatten(),
            attention_mask=source_encoding.attention_mask.flatten(),
            decoder_input_ids=labels.flatten(),
            decoder_attention_mask=target_encoding.attention_mask.flatten(),
        )


class MT5DataModule(pl.LightningDataModule):
    def __init__(
        self,
        tokenizer,
        train_df,
        valid_df,
        test_df,
        batch_size: int = 1,
        num_workers: int = 2,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.valid_df = valid_df
        self.test_df = test_df
        self.tokenizer = tokenizer

    def setup(self, stage: Optional[str] = None, batch_size=1):
        self.batch_size = batch_size
        if stage == "fit" or stage is None:
            self.train_data = MT5Dataset(self.train_df, self.tokenizer)
            self.valid_data = MT5Dataset(self.valid_df, self.tokenizer)

        if stage == "test" or stage is None:
            self.test_data = MT5Dataset(self.test_df, self.tokenizer)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_data, batch_size=self.batch_size, shuffle=True
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.valid_data, batch_size=self.batch_size, shuffle=False
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_data, batch_size=self.batch_size, shuffle=False
        )


class MT5Lightning(pl.LightningModule):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.avg_training_loss = None
        self.avg_val_loss = None

    def forward(
        self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask
    ):
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        decoder_input_ids = batch["decoder_input_ids"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )

        self.log("loss", output[0], prog_bar=True, on_step=True, on_epoch=True)

        return output[0]

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        decoder_input_ids = batch["decoder_input_ids"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )

        self.log("val_loss", output[0], prog_bar=True, on_step=True, on_epoch=True)

        return output[0]

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        decoder_input_ids = batch["decoder_input_ids"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )

        self.log("test_loss", output.loss, prog_bar=True)

        return output.loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=3e-4)

    def training_epoch_end(self, training_step_outputs):
        self.avg_training_loss = np.round(
            torch.mean(torch.stack([x["loss"] for x in training_step_outputs])).item(),
            4,
        )
        path = ""
        if os.path.exists("drive"):
            path += "drive/MyDrive/mt5-thai-qg/"
        else:
            path += "outputs/"
        path += f"mt5-qg-epoch-{self.current_epoch}-train-loss-{self.avg_training_loss}-val-loss-{self.avg_val_loss}"
        self.tokenizer.save_pretrained(path)
        self.model.save_pretrained(path)

    def validation_epoch_end(self, validation_step_outputs):
        _loss = [x.cpu() for x in validation_step_outputs]
        self.avg_val_loss = np.round(
            torch.mean(torch.stack(_loss)).item(),
            4,
        )


Global seed set to 16


## Actually Train
Start actually training

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained(
    "google/mt5-small", return_dict=True
)
tokenizer = MT5TokenizerFast.from_pretrained("google/mt5-small")
dataset = MT5DataModule(tokenizer, train_df, valid_df, test_df, batch_size=1)

MT5Model = MT5Lightning(model, tokenizer)

callbacks = []
callbacks.append(EarlyStopping(monitor="val_loss", mode="min"))

wandb_logger = WandbLogger(
    project="mT5-thai-multiple-e2e-qg", name="mT5-small-thai-multiple-e2e-qg"
)

trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    logger=wandb_logger,
    max_epochs=20,
    log_every_n_steps=1,
    callbacks=callbacks,
    accumulate_grad_batches=20
)

trainer.fit(MT5Model, dataset)


# Testing & Inferencing
Now we will load up our model and do some inference.

## Inferencing

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained(
    "drive/MyDrive/mt5-thai-qg/mt5-qg-epoch-5-train-loss-0.855-val-loss-0.9273",
    return_dict=True,
)
tokenizer = MT5TokenizerFast.from_pretrained(
    "drive/MyDrive/mt5-thai-qg/mt5-qg-epoch-5-train-loss-0.855-val-loss-0.9273"
)

model.cuda()


In [8]:
def predict(text):
    with torch.no_grad():
        input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

        input_ids = input_ids.cuda()

        generated_ids = model.generate(
            input_ids=input_ids,
            num_beams=3,
            max_length=10000,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            top_p=50,
            top_k=20,
            num_return_sequences=1,
        )

        preds = [
            tokenizer.decode(
                g,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )
            for g in generated_ids
        ]
    return preds


Run this cell below to try out the model!

In [20]:
text_to_predict = """สร้าง 5 คำถาม: เฟซบุ๊ก (อังกฤษ: Facebook) เป็นบริการเครือข่ายสังคมสัญชาติอเมริกัน สำนักงานใหญ่อยู่ที่ เมนโลพาร์ก รัฐแคลิฟอร์เนีย เฟซบุ๊กก่อตั้งเมื่อวันพุธที่ 4 กุมภาพันธ์ ค.ศ. 2004 โดยมาร์ก ซักเคอร์เบิร์ก และเพื่อนร่วมห้องภายในมหาวิทยาลัย และเหล่าเพื่อนในมหาวิทยาลัยฮาร์วาร์ด พร้อมโดยสมาชิกเพื่อนผู้ก่อตั้ง Eduardo Saverin, Andrew McCollum, Dustin Moskovitz และ Chris Hughes ในท้ายที่สุดเว็บไซต์มีการเข้าชมอย่างจำกัด ทำให้เหล่านักศึกษาภายในมหาวิทยาลัยฮาร์วาร์ด แต่ภายหลังได้ขยายเพิ่มจำนวนในมหาวิทยาลัย ในพื้นที่บอสตัน ไอวีลีก และมหาวิทยาลัยสแตนฟอร์ด และค่อย ๆ รับรองมหาวิทยาลัยอื่นต่าง ๆ และต่อมาก็รับรองโรงเรียนมัธยมศึกษา โดยเฟซบุ๊กให้การอนุญาตให้เยาวชนอายุต่ำกว่า 13 ปีทั่วโลกสามารถสมัครสมาชิกได้ภายในเว็บไซต์ โดยไม่ต้องอ้างอิงหลักฐานใด ๆ ใน ค.ศ. 2020 เฟซบุ๊กอ้างว่ามีผู้ใช้ที่ยังคงใช้งานรายเดือนที่ 28 พันล้านคน โดยมีผู้ใช้งานทั่วโลกมากเป็นอันดับ 7 และเป็นแอปโทรศัพท์ที่มีคนโหลดมากที่สุดในคริสต์ทศวรรษ 2010"""
predicted = predict(text_to_predict)
predicted


['1. เฟซบุ๊ก เป็นบริการเครือข่ายสังคมสัญชาติอะไร A: อเมริกัน 2. เฟซบุ๊ก ก่อตั้งเมื่อไหร่ A: 4 กุมภาพันธ์ ค.ศ. 2004 3. เฟซบุ๊ก ก่อตั้งโดยใคร A: มาร์ก ซักเคอร์เบิร์ก 4. เฟซบุ๊ก สมาชิกเพื่อนผู้ก่อตั้งคือใคร A: มาร์ก ซักเคอร์เบิร์ก 5. เฟซบุ๊ก มีผู้ใช้ที่ยังคงใช้งานรายเดือนที่เท่าไร A: 28 พันล้านคน']

## Evaluate Our Model

Setup our dataset for testm mode

In [18]:
dataset = MT5DataModule(tokenizer, train_df, valid_df, test_df, batch_size=4)
dataset.setup(stage="test")
test_loader = dataset.test_dataloader()


Define a preprocessing function so that our metrics don't die

In [19]:
from pythainlp import word_tokenize


def pre_process(texts):
    final = []
    for text in texts:
        final.append(" ".join(word_tokenize(text, keep_whitespace=False)))
    return final


Our evaluation will use the following metrics:
* ROUGE
* CHRF
* GLEU
* METEOR

Begin the evaluation!

In [32]:
import nltk
from torchmetrics.text.rouge import ROUGEScore
from torchmetrics import CHRFScore, BLEUScore
from nltk.translate import meteor_score, gleu_score

model.cuda()

nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt")

rouge = ROUGEScore()
chrf = CHRFScore()
bleu = BLEUScore(smooth=True, weights=(0.25, 0.25, 0.25, 0.25))

meteor_avg = 0
gleu_avg = 0

labels_collect = []
preds_collect = []

for batch in test_loader:
    generated_ids = model.generate(
        input_ids=batch["input_ids"].cuda(),
        num_beams=2,
        max_length=1024,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        top_p=50,
        top_k=0.95,
        num_return_sequences=1,
    )

    preds = [
        tokenizer.decode(
            g,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )
        for g in generated_ids
    ]

    batch["decoder_input_ids"][
        batch["decoder_input_ids"] == -100
    ] = tokenizer.pad_token_id

    labels = [
        tokenizer.decode(
            g,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )
        for g in batch["decoder_input_ids"]
    ]

    preds = pre_process(preds)
    labels = pre_process(labels)

    for p, l in zip(preds, labels):
        meteor_avg += meteor_score.single_meteor_score(p.split(" "), l.split(" "))
        gleu_avg += gleu_score.sentence_gleu([l.split(" ")], p.split(" "))

    bleu(preds, [labels])
    chrf(preds, [labels])
    rouge(preds, labels)
    preds_collect.append(preds)
    labels_collect.append(labels)

print("------")
print(f"Meteor: {meteor_avg / (len(test_loader))}")
print(f"GLEU: {gleu_avg / (len(test_loader))}")
print(f"BLEU: {bleu.compute().item()}")
print(f"CHRF: {chrf.compute().item()}")
print(f"ROUGE: {rouge.compute()}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.7/dist-packages/torchmetrics/functional/text/chrf.py:181: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  total_n_grams[n] = tensor(sum(n_grams_counts[n].values()))
/usr/local/lib/python3.7/dist-packages/torchmetrics/functional/text/chrf.py:211: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  for n_gram in hyp_n_grams_counts[n]


------
Meteor: 0.5771521957033449
GLEU: 0.349622574503331
BLEU: 0.3545001149177551
CHRF: 0.4814225435256958
ROUGE: {'rouge1_fmeasure': tensor(0.8742), 'rouge1_precision': tensor(0.9097), 'rouge1_recall': tensor(0.8698), 'rouge2_fmeasure': tensor(0.7027), 'rouge2_precision': tensor(0.7305), 'rouge2_recall': tensor(0.7088), 'rougeL_fmeasure': tensor(0.8554), 'rougeL_precision': tensor(0.8902), 'rougeL_recall': tensor(0.8513), 'rougeLsum_fmeasure': tensor(0.8706), 'rougeLsum_precision': tensor(0.9064), 'rougeLsum_recall': tensor(0.8660)}


# Misc
Below are some other useful chunks of code

## Export prediction & labels

In [33]:
label_final = [i for x in labels_collect for i in x]
pred_final = [i for x in preds_collect for i in x]


export = pd.DataFrame(
    data=zip(label_final, pred_final), columns=["Labels", "Predictions"]
)
export.to_json("output_finetuned_trained.json")


## Zip up Outputs
Zip up the outputs & logs folder (with the models) to prepare them for exporting using zstd.

### Installing dependencies

In [ ]:
!apt install tar zstd

### Zipping up everything

In [ ]:
!tar -c -I "zstd -19 -T0" -f "mt5-thai-qg.tar.zst" outputs/ lightning_logs/